In [ ]:
# Initialization

import math
from datetime import datetime
import openpyxl as opxl
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import model as ty
import proc as pr
import matplotlib.ticker as mtick
import sklearn.metrics

from IPython.core.display import HTML

import heat_transfer as bht

import fluids as fds
import ht 

import general as gen

import os

import scipy.integrate as integrate
import scipy.optimize as sco

import networkx as nx

import plotly.graph_objects as go

import plot_functions_here as pfun

In [ ]:
folder_path = r'G:\Drive partagés\Cercle Hard\Production\R&D - Modèles numériques\PVT-perf-1Dmodel'
geometry_path = folder_path + '\\' + 'Inputs\\Inputs_TEST_V4.5_GG.xlsx'
hypotheses_path = folder_path + '\\' + 'Hypotheses\\Model_hypotheses.xlsx'
condi_path = folder_path + '\\' + 'Steady-state conditions inputs\\221122_TUV_test_conditions_S425-I_MPP.xlsx'

output_name = 'TEST_n1'

componentSpecs = pr.import_geometry(geometry_path)
hyp = pr.create_dict_from_excel(hypotheses_path,'Main')
steadyStateConditions_df = pr.create_inputs_from_excel(condi_path,componentSpecs,hyp)

In [ ]:
# stepConditions = ty.write_stepConditions_from_steadyStateConditions(steadyStateConditions_df,0,hyp)

In [ ]:
stepConditions =   {'G': 0.,
                    'T_amb': 273.15,
                    'T_back': 273.15,
                    'u': 0.00001,
                    'u_back': 0.,
                    'T_fluid_in0': 267.15,
                    'Gp': 4,
                    'T_sky': 273.15,
                    'T_back_rad': 273.15,
                    'mdot': 0.0478,
                    'guess_T_PV': 270.}

In [ ]:
# slices_df,df_one,its_df_list = ty.simu_one_steady_state(par,par_p,hyp)

In [ ]:
df_one,res = ty.simu_one_steady_state_all_he(componentSpecs,stepConditions,hyp)

In [ ]:
for part in componentSpecs['decomp'].keys():
    bal,bal_perc = ty.recap_energy_balances(res[part]['df_one'])
    print(part,bal_perc)
bal,bal_perc = ty.recap_energy_balances(df_one)
print('Total',bal_perc)

In [ ]:
merged_df = pd.concat([df_one] + [res[part]['df_one'] for part in res.keys()], axis=0)
merged_df.insert(0, 'Component', ['Total']+ list(componentSpecs['decomp'].keys()))
merged_df.insert(1, 'Component name', ['Total']+ list(componentSpecs['decomp'].values()))

In [ ]:
pr.disp_html(merged_df)

In [ ]:
df_one['Qdot_tube_fluid']

In [ ]:
A1 = ( - df_one['Qdot_tube_fluid'] / componentSpecs['AG'] / ( (df_one['T_fluid_out'] + df_one['T_fluid_in'])/2 - df_one['T_amb'] ) ).values[0]
print('A1',A1)

In [ ]:
bal,bal_perc = ty.recap_energy_balances(res['main']['df_one'])

In [ ]:
# Create a directed graph
G = nx.DiGraph()

# Add nodes
nodes = ['T_amb', 'T_sky', 'T_glass', 'T_PV', 'T_B', 'T_absfin', 
         'T_tube', 'T_fluid', 'T_ins_absfin', 'T_ins_tube', 'T_back', 'T_back_rad']
G.add_nodes_from(nodes)

# Add edges with labels
edges = [('T_amb', 'T_glass', 'Qdot_top_conv'), ('T_sky', 'T_glass', 'Qdot_top_rad'),
         ('T_glass', 'T_PV', 'Qdot_glass_PV'), ('T_PV', 'T_B', 'Qdot_PV_Base'),
         ('T_PV', 'T_absfin', 'Qdot_PV_absfin'), ('T_absfin', 'T_B', 'Qdot_absfin_Base'),
         ('T_B', 'T_tube', 'Qdot_Base_tube'), ('T_tube', 'T_fluid', 'Qdot_tube_fluid'),
         ('T_absfin', 'T_ins_absfin', 'Qdot_absfin_ins_absfin'),
         ('T_tube', 'T_ins_tube', 'Qdot_tube_ins_tube'),
         ('T_ins_absfin', 'T_back', 'Qdot_ins_absfin_back'),
         ('T_ins_absfin', 'T_back_rad', 'Qdot_ins_absfin_back_rad'),
         ('T_ins_tube', 'T_back', 'Qdot_ins_tube_back'),
         ('T_ins_tube', 'T_back_rad', 'Qdot_ins_tube_back_rad')]

for u, v, label in edges:
    G.add_edge(u, v, label=label)

# Manually set positions based on TikZ coordinates
pos = {
    'T_amb': (3, 0),
    'T_sky': (8, 0),
    'T_glass': (5.5, -3),
    'T_PV': (5.5, -8),
    'T_B': (3, -11),
    'T_absfin': (8, -11),
    'T_tube': (4, -15),
    'T_fluid': (4, -20),
    'T_ins_absfin': (9, -19),
    'T_ins_tube': (6, -19),
    'T_back': (6, -26),
    'T_back_rad': (9, -26)
}

# Draw the graph
nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='black')

# Draw edge labels
edge_labels = nx.get_edge_attributes(G, 'label')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)

plt.show()


In [ ]:
pfun.plot_graph(df_one)

In [ ]:
ht_signs = {'Qdot_sun_PV':1, 'Qdot_sun_glass':1, 'Qdot_top_conv':-1, 'Qdot_top_rad':-1, 'Qdot_PV_sky':-1, 'Qdot_absfin_back_conv':-1, 'Qdot_absfin_back_rad':-1, 'Qdot_Base_back':-1, 'Qdot_tube_back_conv':-1, 'Qdot_tube_back_rad':-1, 'Qdot_tube_sky':-1}
ht_labels = ['Qdot_sun_PV', 'Qdot_sun_glass', 'Qdot_top_conv', 'Qdot_top_rad', 'Qdot_PV_sky', 'Qdot_absfin_back_conv', 'Qdot_absfin_back_rad', 'Qdot_Base_back', 'Qdot_tube_back_conv', 'Qdot_tube_back_rad', 'Qdot_tube_sky']
data = {key:ht_signs[key]*df_one[key].values[0] for key in ht_labels}

# Adjusted approach for a single-column histogram with offset bars
fig = go.Figure()

# Initialize cumulative sums for positive and negative values
cumulative_positive = 0
cumulative_negative = 0

for key, value in data.items():
    if value >= 0:
        # For positive values, start from the last positive cumulative sum
        base = cumulative_positive
        cumulative_positive += value
    else:
        # For negative values, start from the last negative cumulative sum
        base = cumulative_negative
        cumulative_negative += value
    
    fig.add_trace(go.Bar(
        x=["Heat Transfer Contributions"],
        y=[value],
        name=key,
        width=0.2,
        base=base  # Set the starting point of the bar
    ))

# Update the layout
fig.update_layout(
    title="Decomposition of Qdot_tube_fluid in Different Heat Transfer Contributions",
    yaxis_title="Value",
    barmode="overlay"  # Overlay the bars
)

# Show the figure
fig.show()


In [ ]:
125/5.65/1.93